In [3]:
import pandas as pd
from datetime import datetime
import os

# load datasets
assets = {}

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(f"loading {filename}")
        assets[filename] = pd.read_csv(os.path.join(dirname, filename))

loading players.csv
loading competitions.csv
loading games.csv
loading game_events.csv
loading club_games.csv
loading player_valuations.csv
loading appearances.csv
loading clubs.csv


In [4]:
# join assets by their shared IDs
joined = (
    assets['appearances.csv'].merge(
        assets['players.csv'],
        on=['player_id']
    ).merge(
        assets['games.csv'],
        on=['game_id']
    ).merge(
        assets['clubs.csv'],
        left_on='current_club_id',
        right_on='club_id'
    )
)



In [5]:
#select only PL

joined_pl=joined[joined.competition_id_x=='GB1']
joined_pl.head()

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date_x,player_name,competition_id_x,yellow_cards,red_cards,...,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season_y,url
15298,3050324_160380,3050324,160380,931,11127,2018-12-08,Ibrahima Cissé,GB1,0,0,...,27.4,12,48.0,1,Yekaterinburg Arena,23000,€-570k,NaN,2023,https://www.transfermarkt.co.uk/ural-ekaterinb...
15299,3050357_160380,3050357,160380,931,11127,2019-01-01,Ibrahima Cissé,GB1,0,0,...,27.4,12,48.0,1,Yekaterinburg Arena,23000,€-570k,NaN,2023,https://www.transfermarkt.co.uk/ural-ekaterinb...
15301,3050379_160380,3050379,160380,931,11127,2019-01-20,Ibrahima Cissé,GB1,0,0,...,27.4,12,48.0,1,Yekaterinburg Arena,23000,€-570k,NaN,2023,https://www.transfermarkt.co.uk/ural-ekaterinb...
29413,2578168_57420,2578168,57420,289,667,2016-03-05,Lamine Koné,GB1,0,0,...,24.1,13,44.8,7,Stade de la Meinau,26109,€-38.00m,NaN,2023,https://www.transfermarkt.co.uk/rc-strassburg-...
29414,2698012_57420,2698012,57420,289,667,2016-08-27,Lamine Koné,GB1,0,0,...,24.1,13,44.8,7,Stade de la Meinau,26109,€-38.00m,NaN,2023,https://www.transfermarkt.co.uk/rc-strassburg-...


In [6]:
# first step in cleaning..removal of duplicate columns from merge

def drop_columns_by_suffix(dataframe, suffix):
    columns_to_drop = [col for col in dataframe.columns if col.endswith(suffix)]
    dataframe_dropped = dataframe.drop(columns=columns_to_drop)
    return dataframe_dropped

In [7]:
cleaning1=drop_columns_by_suffix(joined_pl,'_y')

In [8]:
cleaning1.drop(columns='url',inplace=True)

In [9]:
cleaning1.head()

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date_x,player_name,competition_id_x,yellow_cards,red_cards,...,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name
15298,3050324_160380,3050324,160380,931,11127,2018-12-08,Ibrahima Cissé,GB1,0,0,...,NaN,25,27.4,12,48.0,1,Yekaterinburg Arena,23000,€-570k,NaN
15299,3050357_160380,3050357,160380,931,11127,2019-01-01,Ibrahima Cissé,GB1,0,0,...,NaN,25,27.4,12,48.0,1,Yekaterinburg Arena,23000,€-570k,NaN
15301,3050379_160380,3050379,160380,931,11127,2019-01-20,Ibrahima Cissé,GB1,0,0,...,NaN,25,27.4,12,48.0,1,Yekaterinburg Arena,23000,€-570k,NaN
29413,2578168_57420,2578168,57420,289,667,2016-03-05,Lamine Koné,GB1,0,0,...,NaN,29,24.1,13,44.8,7,Stade de la Meinau,26109,€-38.00m,NaN
29414,2698012_57420,2698012,57420,289,667,2016-08-27,Lamine Koné,GB1,0,0,...,NaN,29,24.1,13,44.8,7,Stade de la Meinau,26109,€-38.00m,NaN


In [10]:
def rename_columns(dataframe):
    new_columns = [col[:-2] if col.endswith('_x') else col for col in dataframe.columns]
    dataframe.columns = new_columns
    return dataframe

In [11]:
cleaning2=rename_columns(cleaning1)

In [12]:
cleaning2.columns

Index(['appearance_id', 'game_id', 'player_id', 'player_club_id',
       'player_current_club_id', 'date', 'player_name', 'competition_id',
       'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
       'first_name', 'last_name', 'name', 'last_season', 'current_club_id',
       'player_code', 'country_of_birth', 'city_of_birth',
       'country_of_citizenship', 'date_of_birth', 'sub_position', 'position',
       'foot', 'height_in_cm', 'market_value_in_eur',
       'highest_market_value_in_eur', 'contract_expiration_date', 'agent_name',
       'image_url', 'url', 'current_club_domestic_competition_id',
       'current_club_name', 'season', 'round', 'home_club_id', 'away_club_id',
       'home_club_goals', 'away_club_goals', 'home_club_position',
       'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee',
       'home_club_name', 'away_club_name', 'aggregate', 'competition_type',
       'club_id', 'club_co

In [13]:
#removing unnecessary columns

cleaning3=cleaning2[['date', 'player_name', 'current_club_name', 'competition_id',
       'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
       'first_name', 'last_name', 'name', 'last_season','sub_position','date_of_birth', 'position','season', 'round','home_club_goals', 'away_club_goals']]

In [14]:
cleaning3['date_of_birth'] = cleaning3['date_of_birth'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
current_year = pd.Timestamp.now().year
cleaning3['age'] = current_year - cleaning3['date_of_birth'].dt.year

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
#removing players that are no longer in the PL

def delete_rows_by_value_count(dataframe, column_name, threshold):
    # Get the value count for the specified column
    value_counts = dataframe[column_name].value_counts()

    # Find the values with count greater than the threshold
    values_to_keep = value_counts[value_counts > threshold].index

    # Filter the dataframe to keep only the rows with values greater than the threshold
    dataframe_filtered = dataframe[dataframe[column_name].isin(values_to_keep)]

    return dataframe_filtered

In [17]:
cleaning4 = delete_rows_by_value_count(cleaning3, 'current_club_name', 1000)

In [18]:
cleaning5=cleaning4[~cleaning4.current_club_name.isin(['Leeds United','West Bromwich Albion','Norwich City','Southampton FC','Watford FC'])]

In [19]:
cleaning5.current_club_name.value_counts()

Everton FC                 4647
Newcastle United           4144
Crystal Palace             4008
Leicester City             3965
West Ham United            3789
Manchester United          3620
Liverpool FC               3246
Tottenham Hotspur          3192
Chelsea FC                 3059
Manchester City            3010
Aston Villa                2912
Burnley FC                 2850
Brighton & Hove Albion     2455
Arsenal FC                 2303
Nottingham Forest          2216
AFC Bournemouth            2117
Fulham FC                  2053
Wolverhampton Wanderers    1803
Brentford FC               1373
Name: current_club_name, dtype: int64

In [20]:
cleaning6=cleaning5[cleaning5.season==2022]

##### check if total minutes is less than 5 games..then drop

In [21]:
grouped1 = cleaning6.groupby('player_name').minutes_played.sum().reset_index().sort_values('minutes_played',ascending=False)
grouped1

,player_name,minutes_played
2,Aaron Ramsdale,3420
210,James Tarkowski,3420
108,David de Gea,3420
107,David Raya,3420
161,Gabriel Magalhães,3411
...,...,...
106,David Ozoh,1
145,Ethan Nwaneri,1
318,Marquinhos,1
163,George Abbott,1


In [22]:
wipe_list=grouped1[grouped1.minutes_played<400].player_name.to_list()
wipe_list[0:5]

['Elliot Anderson',
 'Ayoze Pérez',
 'Cole Palmer',
 'Carney Chukwuemeka',
 'Calum Chambers']

In [23]:
grouped2=grouped1[~grouped1.player_name.isin(wipe_list)]

In [24]:
print(len(grouped1),len(grouped2))

481 364


In [25]:
cleaning7=cleaning6[cleaning6.player_name.isin(grouped2.player_name.to_list())]

In [26]:
len(cleaning7)

9426

In [27]:
len(cleaning6)

10046

##### ...continue cleaning data

In [28]:
cleaning7['round'].value_counts()

35. Matchday    259
24. Matchday    258
22. Matchday    258
7. Matchday     256
13. Matchday    254
32. Matchday    253
9. Matchday     252
23. Matchday    251
29. Matchday    251
14. Matchday    251
33. Matchday    251
31. Matchday    251
11. Matchday    250
30. Matchday    250
10. Matchday    250
8. Matchday     249
25. Matchday    249
34. Matchday    249
6. Matchday     249
37. Matchday    248
27. Matchday    248
26. Matchday    248
36. Matchday    247
18. Matchday    247
12. Matchday    246
2. Matchday     246
21. Matchday    246
20. Matchday    245
19. Matchday    245
4. Matchday     245
15. Matchday    245
3. Matchday     244
16. Matchday    242
1. Matchday     242
28. Matchday    241
5. Matchday     240
17. Matchday    240
38. Matchday    230
Name: round, dtype: int64

In [29]:
def move_symbol_to_column(df, column_name, symbol, new_column_name):
    new_column = []
    other_column = []
    for value in df[column_name]:
        parts = value.split(symbol, 1)
        if len(parts) == 2:
            new_column.append(parts[0].strip())
            other_column.append(parts[1].strip())
        else:
            new_column.append(value.strip())
            other_column.append('')
    df[new_column_name] = new_column
    df[column_name] = other_column
    return df

In [30]:
cleaning8=move_symbol_to_column(cleaning7,'round','.','wh_round')
cleaning8['round']= cleaning8['wh_round'].astype('int')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [31]:
cleaning8.head()

,date,player_name,current_club_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played,first_name,...,last_season,sub_position,date_of_birth,position,season,round,home_club_goals,away_club_goals,age,wh_round
74423,2022-08-22,Fred,Manchester United,GB1,0,0,0,0,19,NaN,...,2022,Central Midfield,1993-03-05,Midfield,2022,3,2,1,30,3
74424,2022-08-22,Anthony Martial,Manchester United,GB1,0,0,0,1,45,Anthony,...,2022,Centre-Forward,1995-12-05,Attack,2022,3,2,1,28,3
74425,2022-08-22,Cristiano Ronaldo,Manchester United,GB1,0,0,0,0,4,NaN,...,2022,Centre-Forward,1985-02-05,Attack,2022,3,2,1,38,3
74426,2022-08-22,David de Gea,Manchester United,GB1,0,0,0,0,90,David,...,2022,Goalkeeper,1990-11-07,Goalkeeper,2022,3,2,1,33,3
74427,2022-08-22,Christian Eriksen,Manchester United,GB1,0,0,0,0,86,Christian,...,2022,Central Midfield,1992-02-14,Midfield,2022,3,2,1,31,3


In [32]:
cleaning8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9426 entries, 74423 to 1111037
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               9426 non-null   object        
 1   player_name        9426 non-null   object        
 2   current_club_name  9426 non-null   object        
 3   competition_id     9426 non-null   object        
 4   yellow_cards       9426 non-null   int64         
 5   red_cards          9426 non-null   int64         
 6   goals              9426 non-null   int64         
 7   assists            9426 non-null   int64         
 8   minutes_played     9426 non-null   int64         
 9   first_name         8676 non-null   object        
 10  last_name          9426 non-null   object        
 11  name               9426 non-null   object        
 12  last_season        9426 non-null   int64         
 13  sub_position       9426 non-null   object        
 14  d

In [33]:
cleaning8['goals_per_min']=cleaning8.goals/cleaning8.minutes_played
cleaning8['ass_per_min']=cleaning8.assists/cleaning8.minutes_played

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
cleaning8.head()

,date,player_name,current_club_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played,first_name,...,date_of_birth,position,season,round,home_club_goals,away_club_goals,age,wh_round,goals_per_min,ass_per_min
74423,2022-08-22,Fred,Manchester United,GB1,0,0,0,0,19,NaN,...,1993-03-05,Midfield,2022,3,2,1,30,3,0.0,0.000000
74424,2022-08-22,Anthony Martial,Manchester United,GB1,0,0,0,1,45,Anthony,...,1995-12-05,Attack,2022,3,2,1,28,3,0.0,0.022222
74425,2022-08-22,Cristiano Ronaldo,Manchester United,GB1,0,0,0,0,4,NaN,...,1985-02-05,Attack,2022,3,2,1,38,3,0.0,0.000000
74426,2022-08-22,David de Gea,Manchester United,GB1,0,0,0,0,90,David,...,1990-11-07,Goalkeeper,2022,3,2,1,33,3,0.0,0.000000
74427,2022-08-22,Christian Eriksen,Manchester United,GB1,0,0,0,0,86,Christian,...,1992-02-14,Midfield,2022,3,2,1,31,3,0.0,0.000000


In [35]:
cleaning8.groupby(['player_name','current_club_name','position','age']).goals_per_min.sum().nlargest(20).sort_values(ascending=False)

player_name           current_club_name  position  age
Erling Haaland        Manchester City    Attack    23     0.495393
Callum Wilson         Newcastle United   Attack    31     0.365262
Roberto Firmino       Liverpool FC       Attack    32     0.348272
Harry Kane            Tottenham Hotspur  Attack    30     0.335043
Yoane Wissa           Brentford FC       Attack    27     0.322614
Michail Antonio       West Ham United    Attack    33     0.306801
Jean-Philippe Mateta  Crystal Palace     Attack    26     0.300000
Jaidon Anthony        AFC Bournemouth    Attack    24     0.254589
Darwin Núñez          Liverpool FC       Attack    24     0.230986
João Félix            Chelsea FC         Attack    24     0.227867
Ivan Toney            Brentford FC       Attack    27     0.223306
Phil Foden            Manchester City    Attack    23     0.223217
Mohamed Salah         Liverpool FC       Attack    31     0.217549
Manor Solomon         Fulham FC          Attack    24     0.216111
Marcus 

In [36]:
cleaning8.groupby(['player_name','current_club_name','position','age']).ass_per_min.sum().nlargest(20).sort_values(ascending=False)

player_name            current_club_name        position  age
Pedro Neto             Wolverhampton Wanderers  Attack    23     1.000000
Phil Foden             Manchester City          Attack    23     0.395019
Amadou Onana           Everton FC               Midfield  22     0.346847
Josh Dasilva           Brentford FC             Midfield  25     0.345528
Kevin De Bruyne        Manchester City          Midfield  32     0.222093
Hakim Ziyech           Chelsea FC               Attack    30     0.207665
Ivan Perisic           Tottenham Hotspur        Midfield  34     0.204247
Leandro Trossard       Brighton & Hove Albion   Attack    29     0.188054
Christian Pulisic      Chelsea FC               Attack    25     0.166667
Patson Daka            Leicester City           Attack    25     0.163699
Bukayo Saka            Arsenal FC               Attack    22     0.149485
Pablo Fornals          West Ham United          Midfield  27     0.142857
Roberto Firmino        Liverpool FC             At

#### create  scorepoints according to position and then see per_min quotes...

In [37]:
cleaning8.head()

,date,player_name,current_club_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played,first_name,...,date_of_birth,position,season,round,home_club_goals,away_club_goals,age,wh_round,goals_per_min,ass_per_min
74423,2022-08-22,Fred,Manchester United,GB1,0,0,0,0,19,NaN,...,1993-03-05,Midfield,2022,3,2,1,30,3,0.0,0.000000
74424,2022-08-22,Anthony Martial,Manchester United,GB1,0,0,0,1,45,Anthony,...,1995-12-05,Attack,2022,3,2,1,28,3,0.0,0.022222
74425,2022-08-22,Cristiano Ronaldo,Manchester United,GB1,0,0,0,0,4,NaN,...,1985-02-05,Attack,2022,3,2,1,38,3,0.0,0.000000
74426,2022-08-22,David de Gea,Manchester United,GB1,0,0,0,0,90,David,...,1990-11-07,Goalkeeper,2022,3,2,1,33,3,0.0,0.000000
74427,2022-08-22,Christian Eriksen,Manchester United,GB1,0,0,0,0,86,Christian,...,1992-02-14,Midfield,2022,3,2,1,31,3,0.0,0.000000


In [38]:
cleaning8.position.value_counts()

Defender      3245
Midfield      2928
Attack        2585
Goalkeeper     668
Name: position, dtype: int64

In [39]:
def scor_calc(df):
    df['scorerpoints']=0
    for index, row in df.iterrows():
        if row['position']=='Midfield':
            df.loc[index, 'scorerpoints'] = 5*row['goals']+3*row['goals']
        elif row['position']=='Defender':
            df.loc[index, 'scorerpoints'] = 6*row['goals']+3*row['goals']
        elif row['position']=='Attack':
            df.loc[index, 'scorerpoints'] = 4*row['goals']+3*row['goals']
    return df   

In [40]:
main = scor_calc(cleaning8)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [41]:
main['score_pts_per_min']=main['scorerpoints']/main['minutes_played']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
main[main.scorerpoints!=0]

,date,player_name,current_club_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played,first_name,...,season,round,home_club_goals,away_club_goals,age,wh_round,goals_per_min,ass_per_min,scorerpoints,score_pts_per_min
74431,2022-08-22,Marcus Rashford,Manchester United,GB1,0,0,1,0,86,Marcus,...,2022,3,2,1,26,3,0.011628,0.000000,7,0.081395
74433,2022-08-22,Jadon Sancho,Manchester United,GB1,0,0,1,0,71,Jadon,...,2022,3,2,1,23,3,0.014085,0.000000,7,0.098592
74582,2023-04-27,Marcus Rashford,Manchester United,GB1,0,0,1,1,90,Marcus,...,2022,33,2,2,26,33,0.011111,0.011111,7,0.077778
74583,2023-04-27,Jadon Sancho,Manchester United,GB1,0,0,1,0,61,Jadon,...,2022,33,2,2,23,33,0.016393,0.000000,7,0.114754
74676,2022-09-04,Marcus Rashford,Manchester United,GB1,0,0,2,1,80,Marcus,...,2022,6,3,1,26,6,0.025000,0.012500,14,0.175000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111011,2023-01-19,Dejan Kulusevski,Tottenham Hotspur,GB1,0,0,1,0,90,Dejan,...,2022,7,4,2,23,7,0.011111,0.000000,7,0.077778
1111012,2023-01-19,Emerson Royal,Tottenham Hotspur,GB1,0,0,1,0,78,NaN,...,2022,7,4,2,24,7,0.012821,0.000000,9,0.115385
1111015,2023-02-05,Harry Kane,Tottenham Hotspur,GB1,0,0,1,0,90,Harry,...,2022,22,1,0,30,22,0.011111,0.000000,7,0.077778
1111028,2023-04-08,Heung-min Son,Tottenham Hotspur,GB1,0,0,1,0,90,Heung-min,...,2022,30,2,1,31,30,0.011111,0.000000,7,0.077778


In [43]:
main.groupby(['player_name','age']).agg({'goals_per_min':'mean','ass_per_min':'mean','score_pts_per_min':['mean','sum']}).sort_values(('score_pts_per_min','sum'), ascending=False)

goals_per_min ass_per_min score_pts_per_min  \
                                   mean        mean              mean   
player_name           age                                               
Erling Haaland        23       0.014154    0.002926          0.099079   
Callum Wilson         31       0.011783    0.002193          0.082478   
Roberto Firmino       32       0.013931    0.005375          0.097516   
Harry Kane            30       0.008817    0.000877          0.061718   
Yoane Wissa           27       0.008490    0.000968          0.059429   
...                                 ...         ...               ...   
Kevin Schade          22       0.000000    0.004274          0.000000   
Keylor Navas          37       0.000000    0.000000          0.000000   
Kieran Tierney        26       0.000000    0.001481          0.000000   
Konstantinos Tsimikas 27       0.000000    0.005935          0.000000   
Álex Moreno           30       0.000000    0.002082          0.000000   

                                     
                                sum  
player_name           age            
Erling Haaland        23   3.467748  
Callum Wilson         31   2.556832  
Roberto Firmino       32   2.437903  
Harry Kane            30   2.345299  
Yoane Wissa           27   2.258297  
...                             ...  
Kevin Schade          22   0.000000  
Keylor Navas          37   0.000000  
Kieran Tierney        26   0.000000  
Konstantinos Tsimikas 27   0.000000  
Álex Moreno           30   0.000000  

[364 rows x 4 columns]

In [44]:
main.groupby(['player_name','current_club_name','position','age']).score_pts_per_min.sum().nlargest(20).sort_values(ascending=False)

player_name           current_club_name  position  age
Erling Haaland        Manchester City    Attack    23     3.467748
Callum Wilson         Newcastle United   Attack    31     2.556832
Roberto Firmino       Liverpool FC       Attack    32     2.437903
Harry Kane            Tottenham Hotspur  Attack    30     2.345299
Yoane Wissa           Brentford FC       Attack    27     2.258297
Michail Antonio       West Ham United    Attack    33     2.147605
Jean-Philippe Mateta  Crystal Palace     Attack    26     2.100000
Jaidon Anthony        AFC Bournemouth    Attack    24     1.782126
Darwin Núñez          Liverpool FC       Attack    24     1.616901
João Félix            Chelsea FC         Attack    24     1.595071
Ivan Toney            Brentford FC       Attack    27     1.563144
Phil Foden            Manchester City    Attack    23     1.562517
Mohamed Salah         Liverpool FC       Attack    31     1.522840
Manor Solomon         Fulham FC          Attack    24     1.512778
Marcus 

In [45]:
scorer_grouped = main.groupby(['player_name','current_club_name','position','age']).agg({'score_pts_per_min':'sum','minutes_played':'sum'}).sort_values(by='score_pts_per_min',ascending=False).reset_index()
scorer_grouped.head(10)

,player_name,current_club_name,position,age,score_pts_per_min,minutes_played
0,Erling Haaland,Manchester City,Attack,23,3.467748,2777
1,Callum Wilson,Newcastle United,Attack,31,2.556832,1887
2,Roberto Firmino,Liverpool FC,Attack,32,2.437903,1212
3,Harry Kane,Tottenham Hotspur,Attack,30,2.345299,3408
4,Yoane Wissa,Brentford FC,Attack,27,2.258297,1600
5,Michail Antonio,West Ham United,Attack,33,2.147605,1838
6,Jean-Philippe Mateta,Crystal Palace,Attack,26,2.100000,754
7,Jaidon Anthony,AFC Bournemouth,Attack,24,1.782126,1247
8,Darwin Núñez,Liverpool FC,Attack,24,1.616901,1698
9,João Félix,Chelsea FC,Attack,24,1.595071,942


In [46]:
scorer_grouped[scorer_grouped.position=='Defender'].head(20)

,player_name,current_club_name,position,age,score_pts_per_min,minutes_played
67,Ben Chilwell,Chelsea FC,Defender,27,0.623288,1408
109,Gabriel Magalhães,Arsenal FC,Defender,26,0.300000,3411
110,Ben White,Arsenal FC,Defender,26,0.300000,3067
111,Virgil van Dijk,Liverpool FC,Defender,32,0.300000,2835
112,John Stones,Manchester City,Defender,29,0.300000,1848
113,Ethan Pinnock,Brentford FC,Defender,30,0.300000,2700
114,Ben Mee,Brentford FC,Defender,34,0.300000,3272
118,Rayan Aït-Nouri,Wolverhampton Wanderers,Defender,22,0.290323,1070
121,Matías Viña,AFC Bournemouth,Defender,26,0.283673,455
125,Marcos Senesi,AFC Bournemouth,Defender,26,0.269811,2495


In [47]:
scorer_grouped[scorer_grouped.position=='Midfield'].head(20)

,player_name,current_club_name,position,age,score_pts_per_min,minutes_played
18,Martin Ødegaard,Arsenal FC,Midfield,25,1.385948,3150
26,James Maddison,Leicester City,Midfield,27,1.162992,2486
29,Eberechi Eze,Crystal Palace,Midfield,25,1.115516,2645
31,Vitaly Janelt,Brentford FC,Midfield,25,1.023954,2210
35,Kai Havertz,Chelsea FC,Midfield,24,0.982911,2579
37,Douglas Luiz,Aston Villa,Midfield,25,0.923705,2932
41,Alexis Mac Allister,Brighton & Hove Albion,Midfield,25,0.888889,2886
45,Pascal Groß,Brighton & Hove Albion,Midfield,32,0.800000,3246
47,Josh Dasilva,Brentford FC,Midfield,25,0.773166,1383
48,Ilkay Gündogan,Manchester City,Midfield,33,0.772650,2359
